# 💨 Smart Cage - MQ2 Gas Sensor Dataset Collection

**Samsung Innovation Campus - Phase 3**

Mengumpulkan data dari MQ2 Gas Sensor via MQTT.

**Logika Labeling:**
- **Aman**: MQ2 LOW (tidak ada gas)
- **Waspada**: MQ2 HIGH (gas terdeteksi)
- **Bahaya**: MQ2 HIGH + Suhu > 55°C (kebakaran)

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install paho-mqtt pandas

: 

## 📚 Step 2: Import Libraries

In [ ]:
import json
import pandas as pd
import paho.mqtt.client as mqtt
from datetime import datetime
import time

print("✅ Libraries imported!")

## ⚙️ Step 3: MQTT Configuration

In [ ]:
# MQTT Config
MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 1883
TOPIC_GAS_DATA = "final-project/Mahasiswa-Berpola-Pikir/smartcage/gas/data"

# Collection settings
target_samples = 150
data_collected = []

print(f"📡 Broker: {MQTT_BROKER}")
print(f"📍 Topic: {TOPIC_GAS_DATA}")
print(f"🎯 Target: {target_samples} samples")

## 🎯 Step 4: Auto-Labeling Function

In [ ]:
def auto_label(gas_detected, temp):
    """
    Auto-label berdasarkan kondisi:
    - Aman: MQ2 LOW (gas_detected = false)
    - Waspada: MQ2 HIGH (gas_detected = true)
    - Bahaya: MQ2 HIGH + temp > 55°C
    """
    if not gas_detected:
        return "Aman"
    elif gas_detected and temp > 55:
        return "Bahaya"
    else:
        return "Waspada"

# Test labeling
print("Test auto-labeling:")
print(f"  gas=False, temp=30 → {auto_label(False, 30)}")
print(f"  gas=True, temp=30  → {auto_label(True, 30)}")
print(f"  gas=True, temp=60  → {auto_label(True, 60)}")

## 📡 Step 5: MQTT Callbacks

In [ ]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("✅ Connected to MQTT broker")
        client.subscribe(TOPIC_GAS_DATA)
        print(f"📍 Subscribed to: {TOPIC_GAS_DATA}")
    else:
        print(f"❌ Connection failed with code {rc}")

def on_message(client, userdata, msg):
    global data_collected
    try:
        payload = msg.payload.decode('utf-8')
        data = json.loads(payload)
        
        # Extract data
        gas_detected = data.get('gas_detected', False)
        temp = data.get('temp', 0)
        
        # Auto-label
        label = auto_label(gas_detected, temp)
        
        # Create entry
        entry = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'gas_detected': 1 if gas_detected else 0,
            'temp': temp,
            'label': label
        }
        data_collected.append(entry)
        
        # Progress
        count = len(data_collected)
        print(f"[{count}/{target_samples}] Gas={gas_detected}, Temp={temp:.1f}°C → {label}")
        
        # Auto-stop
        if count >= target_samples:
            print(f"\n✅ Target {target_samples} samples reached!")
            client.disconnect()
    except Exception as e:
        print(f"❌ Error: {e}")

print("✅ Callbacks defined!")

## 🚀 Step 6: Start Collection

**Tips untuk data seimbang:**
1. Biarkan berjalan normal untuk data "Aman"
2. Expose MQ2 ke gas (korek api) untuk data "Waspada"
3. Untuk "Bahaya", simulasikan dengan memanaskan DHT11 atau edit data manual

In [ ]:
# Reset data
data_collected = []

# Create client
client = mqtt.Client(client_id=f"MQ2Collector_{int(time.time())}")
client.on_connect = on_connect
client.on_message = on_message

# Connect
print(f"🔌 Connecting to {MQTT_BROKER}...")
client.connect(MQTT_BROKER, MQTT_PORT, 60)

# Run loop
print("📡 Collecting data...")
client.loop_forever()

## 💾 Step 7: Save to CSV

In [ ]:
if len(data_collected) > 0:
    df = pd.DataFrame(data_collected)
    
    # Save CSV
    filename = f"mq2_gas_dataset_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(filename, index=False)
    print(f"✅ Saved: {filename}")
    print(f"📊 Total: {len(df)} samples")
    
    # Distribution
    print("\n📈 Label Distribution:")
    print(df['label'].value_counts())
    
    # Preview
    print("\n📋 Preview:")
    display(df.head(10))
else:
    print("⚠️ No data collected!")